# 多只股票RSI底背离分析

本Notebook将分析贵州茅台、中芯国际、东方财富和中兴通讯的RSI底背离信号。

In [1]:
import sys
sys.path.append('.')

import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 设置tushare token
TUSHARE_TOKEN = os.getenv('TUSHARE_TOKEN')
if not TUSHARE_TOKEN:
    raise ValueError('请设置TUSHARE_TOKEN环境变量')

In [2]:
# 导入自定义模块
try:
    import tushare as ts
    from signals.calculator import calculate_rsi, detect_rsi_divergence
    ts.set_token(TUSHARE_TOKEN)
    pro = ts.pro_api()
except ImportError as e:
    print(f'导入模块时出错: {e}')
    exit(1)

In [3]:
def get_stock_data(stock_code, end_date, days=365):
    """
    获取股票数据
    """
    start_date = (datetime.strptime(end_date, '%Y%m%d') - timedelta(days=days)).strftime('%Y%m%d')
    df = ts.pro_bar(ts_code=stock_code, adj='qfq', start_date=start_date, end_date=end_date)
    if df is not None and not df.empty:
        df['trade_date'] = pd.to_datetime(df['trade_date'])
        df.set_index('trade_date', inplace=True)
        df.sort_index(inplace=True)
        return df
    return None

def analyze_stock_rsi_divergence(stock_code, stock_name, end_date):
    """
    分析单只股票的RSI底背离信号
    """
    print(f'正在分析 {stock_name} ({stock_code})...')
    
    # 获取股票数据
    df = get_stock_data(stock_code, end_date)
    if df is None or df.empty:
        print(f'无法获取 {stock_name} 的数据')
        return []
    
    # 计算RSI
    rsi_period = 14
    df['rsi'] = calculate_rsi(df, rsi_period)
    
    # 检测RSI底背离信号
    divergence_signals = detect_rsi_divergence(df, df['rsi'], rsi_period)
    
    # 提取底背离信号
    bullish_divergence = divergence_signals['rsi_bullish_divergence']
    
    # 筛选信号日期
    signals_df = df[bullish_divergence]
    
    # 准备结果数据
    results = []
    for date, row in signals_df.iterrows():
        results.append({
            'stock_code': stock_code,
            'stock_name': stock_name,
            'date': date.strftime('%Y-%m-%d'),
            'close_price': row['close'],
            'rsi_value': row['rsi']
        })
    
    print(f'{stock_name} 发现 {len(results)} 个RSI底背离信号')
    return results

In [4]:
# 定义股票信息
stocks = {
    '600519.SH': '贵州茅台',
    '688981.SH': '中芯国际',
    '300059.SZ': '东方财富',
    '000063.SZ': '中兴通讯'
}

# 设置分析日期
end_date = datetime.now().strftime('%Y%m%d')
print(f'分析日期: {end_date}')

# 存储所有结果
all_results = []

# 分析每只股票
for stock_code, stock_name in stocks.items():
    results = analyze_stock_rsi_divergence(stock_code, stock_name, end_date)
    all_results.extend(results)

    # 创建结果DataFrame
    if all_results:
        results_df = pd.DataFrame(all_results)
        print(f'总共发现 {len(all_results)} 个RSI底背离信号')
        print(results_df.head(20))
        
        # 保存结果到CSV文件
        filename = f'multiple_stocks_rsi_bullish_divergence_{end_date}.csv'
        results_df.to_csv(filename, index=False)
        print(f'结果已保存到 {filename}')
    else:
        print('未发现任何RSI底背离信号')

分析日期: 20250828
正在分析 贵州茅台 (600519.SH)...


贵州茅台 发现 20 个RSI底背离信号
总共发现 20 个RSI底背离信号
   stock_code stock_name        date  close_price  rsi_value
0   600519.SH       贵州茅台  2024-11-15      1505.50  48.864572
1   600519.SH       贵州茅台  2025-01-22      1413.31  37.288323
2   600519.SH       贵州茅台  2025-01-24      1408.40  48.626354
3   600519.SH       贵州茅台  2025-01-27      1407.41  48.223844
4   600519.SH       贵州茅台  2025-02-05      1376.82  38.976652
5   600519.SH       贵州茅台  2025-02-06      1385.69  41.491257
6   600519.SH       贵州茅台  2025-02-10      1404.00  46.798597
7   600519.SH       贵州茅台  2025-02-11      1390.75  34.840320
8   600519.SH       贵州茅台  2025-02-12      1415.27  43.053955
9   600519.SH       贵州茅台  2025-02-17      1443.34  49.142857
10  600519.SH       贵州茅台  2025-04-24      1522.42  45.952653
11  600519.SH       贵州茅台  2025-04-30      1517.27  48.900606
12  600519.SH       贵州茅台  2025-06-24      1409.58  24.371498
13  600519.SH       贵州茅台  2025-06-25      1408.27  21.944790
14  600519.SH       贵州茅台  2025-06-26      1420

中芯国际 发现 27 个RSI底背离信号
总共发现 47 个RSI底背离信号
   stock_code stock_name        date  close_price  rsi_value
0   600519.SH       贵州茅台  2024-11-15      1505.50  48.864572
1   600519.SH       贵州茅台  2025-01-22      1413.31  37.288323
2   600519.SH       贵州茅台  2025-01-24      1408.40  48.626354
3   600519.SH       贵州茅台  2025-01-27      1407.41  48.223844
4   600519.SH       贵州茅台  2025-02-05      1376.82  38.976652
5   600519.SH       贵州茅台  2025-02-06      1385.69  41.491257
6   600519.SH       贵州茅台  2025-02-10      1404.00  46.798597
7   600519.SH       贵州茅台  2025-02-11      1390.75  34.840320
8   600519.SH       贵州茅台  2025-02-12      1415.27  43.053955
9   600519.SH       贵州茅台  2025-02-17      1443.34  49.142857
10  600519.SH       贵州茅台  2025-04-24      1522.42  45.952653
11  600519.SH       贵州茅台  2025-04-30      1517.27  48.900606
12  600519.SH       贵州茅台  2025-06-24      1409.58  24.371498
13  600519.SH       贵州茅台  2025-06-25      1408.27  21.944790
14  600519.SH       贵州茅台  2025-06-26      1420

东方财富 发现 13 个RSI底背离信号
总共发现 60 个RSI底背离信号
   stock_code stock_name        date  close_price  rsi_value
0   600519.SH       贵州茅台  2024-11-15      1505.50  48.864572
1   600519.SH       贵州茅台  2025-01-22      1413.31  37.288323
2   600519.SH       贵州茅台  2025-01-24      1408.40  48.626354
3   600519.SH       贵州茅台  2025-01-27      1407.41  48.223844
4   600519.SH       贵州茅台  2025-02-05      1376.82  38.976652
5   600519.SH       贵州茅台  2025-02-06      1385.69  41.491257
6   600519.SH       贵州茅台  2025-02-10      1404.00  46.798597
7   600519.SH       贵州茅台  2025-02-11      1390.75  34.840320
8   600519.SH       贵州茅台  2025-02-12      1415.27  43.053955
9   600519.SH       贵州茅台  2025-02-17      1443.34  49.142857
10  600519.SH       贵州茅台  2025-04-24      1522.42  45.952653
11  600519.SH       贵州茅台  2025-04-30      1517.27  48.900606
12  600519.SH       贵州茅台  2025-06-24      1409.58  24.371498
13  600519.SH       贵州茅台  2025-06-25      1408.27  21.944790
14  600519.SH       贵州茅台  2025-06-26      1420

中兴通讯 发现 17 个RSI底背离信号
总共发现 77 个RSI底背离信号
   stock_code stock_name        date  close_price  rsi_value
0   600519.SH       贵州茅台  2024-11-15      1505.50  48.864572
1   600519.SH       贵州茅台  2025-01-22      1413.31  37.288323
2   600519.SH       贵州茅台  2025-01-24      1408.40  48.626354
3   600519.SH       贵州茅台  2025-01-27      1407.41  48.223844
4   600519.SH       贵州茅台  2025-02-05      1376.82  38.976652
5   600519.SH       贵州茅台  2025-02-06      1385.69  41.491257
6   600519.SH       贵州茅台  2025-02-10      1404.00  46.798597
7   600519.SH       贵州茅台  2025-02-11      1390.75  34.840320
8   600519.SH       贵州茅台  2025-02-12      1415.27  43.053955
9   600519.SH       贵州茅台  2025-02-17      1443.34  49.142857
10  600519.SH       贵州茅台  2025-04-24      1522.42  45.952653
11  600519.SH       贵州茅台  2025-04-30      1517.27  48.900606
12  600519.SH       贵州茅台  2025-06-24      1409.58  24.371498
13  600519.SH       贵州茅台  2025-06-25      1408.27  21.944790
14  600519.SH       贵州茅台  2025-06-26      1420